In [1]:
# 授权绑定Google Drive 
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools 
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null 
!apt-get update -qq 2>&1 > /dev/null 
!apt-get -y install -qq google-drive-ocamlfuse fuse 
from google.colab import auth 
auth.authenticate_user() 
from oauth2client.client import GoogleCredentials 
creds = GoogleCredentials.get_application_default() 
import getpass 
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL 
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}


E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 130812 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.3-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.3-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.3-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [2]:
# 指定Google Drive云端硬盘的根目录，名为drive
!mkdir -p drive
!google-drive-ocamlfuse drive

# 指定当前的工作目录
import os

# 此处为google drive中的文件路径,drive为之前指定的工作根目录，要加上
os.chdir("drive") 


# 查看文件目录，是否包含所需的文件
!ls

 2018_FALL			   Intern
'2110 Lecture Notes.zip'	   l03_scheduling.pdf
'499款简历模板 (df879040).zip'	   lec4820
 499款简历模板.zip		   Log_Analysis.zip
'Academic OneFile'		   Notability
 Acoustic_Localization		   OldKernel_Cyclictest_loaded_unloaded.ods
'cheat sheet.odt'		  'Personal Info'
 Coding_BOOK			  'Project Plan.odt'
'Colab Notebooks'		   Proposal_ECE5720.odt
 CS_4820			  'Robotics Introduction.pdf'
'CS4820-lecture note.zip'	   row_data_ID.csv
 CS5412_Cloud_Computing		   tututut.pdf
 CUDA-CNN-master		   Untitled0.ipynb
'data_Emre.zip (Unzipped Files)'  'Untitled presentation.pdf'
 datathon_report.odt		  'Untitled presentation.pdf (be7cc8da)'
 ECE4450_computerNetwork	   VID_20181207_155103.mp4
 ECE4950_pattern		  'Wu, Yanling - Bursar Account Activity.pdf'
' Intermediate Report.odt'


In [0]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


In [0]:
#Read two kinds of data from file
data = pd.read_excel('./CS5412_Cloud_Computing/Digital Agriculture/Copy of Repro all cows.XLS')
data_afi = pd.read_excel('./CS5412_Cloud_Computing/Digital Agriculture/AfiFarm_Daily data_Walnut Ridge_20180819 (April-Aug).xlsx')

data.head(5)


,ID,FDAT,LACT,bolus,DATB1,First Breeding,DIMB1,CODB1,OUTB1,DATB2,...,ABDIM2,ABDAT3,ABDIM3,ABDAT4,ABDIM4,ABDAT5,DCCPR1,DCCPR2,DCCPR3,DCCPR4
0,1,2018-06-04,3,.,2018-08-09,Thu,66,T,O,2018-09-04 00:00:00,...,.,.,.,.,.,.,34DAYS,62DAYS,.,.
1,2,2018-06-04,2,.,2018-08-09,Thu,66,T,O,2018-08-31 00:00:00,...,.,.,.,.,.,.,31DAYS,66DAYS,.,.
2,3,2018-09-17,2,.,2018-11-22,Thu,66,T,O,2018-12-17 00:00:00,...,.,.,.,.,.,.,31DAYS,.,.,.
3,10,2018-12-28,3,1700003117G8V,2019-03-07,Thu,69,T,O,2019-03-14 00:00:00,...,.,.,.,.,.,.,.,.,.,.
4,18,2018-10-09,3,.,2018-12-13,Thu,65,T,P,.,...,.,.,.,.,.,.,32DAYS,60DAYS,.,.


In [0]:
#get unique list of ID
ID = data["ID"].unique()
ID.shape
data["First Breeding"][0]

'Thu'

In [0]:
#get the unique list of ID of afi dataset
data_afi.head(5)
id_afi = data_afi["Animal_ID"].unique()
id_afi.shape
col = data.columns

li = []
for s in col:
    li.append(data[s])
len(li[0])

data.iloc[0:3,:].shape
# data_afi["Animal_ID"]


(3, 80)

In [0]:
#get the dictionary of the all cow; key is ID, value is all other info
data_dic = {}
for i in range(ID.shape[0]): 
    data_dic[ID[i]] = [data.iloc[i,:]]


In [0]:
#Accoding the ID, append the info of cow into the dictionary
for i in range(data_afi["Animal_ID"].shape[0]):
    if data_afi["Animal_ID"][i] in ID:
        data_dic[data_afi["Animal_ID"][i]].append(data_afi.iloc[i,:])

In [0]:
# split the pregenency info of cow into two lists , listp & listo

def findData(list1, dic, key):
    ind = 8
    listp = []
    listo = []
    while(ind < len(list1)):
        if list1[ind] == 'P':
            listp.append(list1[ind-4])
            break
        elif list1[ind] == 'O':
            listo.append(list1[ind-4])
            ind += 5
        else:
            break
    dic[key] = [listp, listo]
#set the threshold that the data is within 10 days are shared smae label
threshold = datetime.datetime(2019, 1, 22, 0, 0) - datetime.datetime(2019, 1, 12, 0, 0)    
time_dic = {}
for key in data_dic:
    findData(data_dic[key][0],time_dic,key)

                    
print((time_dic[3]))


        

[[datetime.datetime(2019, 2, 15, 0, 0)], [Timestamp('2018-11-22 00:00:00'), datetime.datetime(2018, 12, 17, 0, 0), datetime.datetime(2019, 1, 22, 0, 0)]]


In [0]:
# print(time_dic[300])
count = 0
def createSet(time_list, afi_table, threshold, dataset):  
    for li in afi_table:     
        if time_list[0] != [] and abs(li[2] - time_list[0][0]) < threshold: 
            dataset.append([li[:],1])
        else: 
            for time_o in time_list[1]:
                if abs(li[2] - time_o) < threshold: 
                    dataset.append([li[:],0])
                    break
                
dataset = []
# createSet(time_dic[1],data_dic[1][1:],threshold, dataset)
for key in time_dic:
    createSet(time_dic[key],data_dic[key][1:],threshold, dataset)
    count += 1
    if (count % 500 == 0):
        print(count)
print(len(dataset))

500
1000
1500
14132


In [0]:
print((dataset[0]))  

[Animal_ID                                  1
Group_ID                                  22
Date                     2018-08-01 00:00:00
Dry_Days                                   0
Days_in_Milk                              58
Is_Milked                                  1
Lactation_Num                              3
Yield (gr)                             52840
Conductivity                              93
Fat (%)                                 3.32
Protein (%)                             2.96
Lactose (%)                              4.5
Blood (%)                               0.03
SCC (*1000/ml)                         56624
Activity (steps/hour)                    139
RestTime (minutes)                       613
RestBout (#)                              12
ProdRate (gr/hour)                      2201
RestRatio (%)                             43
RestRestlessness                          26
RestPerBout (minutes)                     51
ActivityDeviation (%)                     -4
Name: 14,

In [0]:
import csv
import numpy as np

print(type(dataset))    


data_new = []
for data in dataset:
    temp = []
    for p in data[0]:
        temp.append(p)
    temp.append(data[1])
    data_new.append(temp)
print(len(data_new), len(data_new[0]))
data_set_new = pd.DataFrame(data_new)


<class 'list'>
14132 23


In [0]:
columns_afi = data_afi.columns
columns_afi = columns_afi[3:]
print(type(columns_afi))
columns = ['Animal_ID','Group_ID','Date','Dry_Days', 'Days_in_Milk', 'Is_Milked', 'Lactation_Num', 'Yield (gr)',
       'Conductivity', 'Fat (%)', 'Protein (%)', 'Lactose (%)', 'Blood (%)',
       'SCC (*1000/ml)', 'Activity (steps/hour)', 'RestTime (minutes)',
       'RestBout (#)', 'ProdRate (gr/hour)', 'RestRatio (%)',
       'RestRestlessness', 'RestPerBout (minutes)', 'ActivityDeviation (%)','label']
data_set_new.columns = columns

<class 'pandas.core.indexes.base.Index'>


In [0]:
data_set_new.head()

,Animal_ID,Group_ID,Date,Dry_Days,Days_in_Milk,Is_Milked,Lactation_Num,Yield (gr),Conductivity,Fat (%),...,SCC (*1000/ml),Activity (steps/hour),RestTime (minutes),RestBout (#),ProdRate (gr/hour),RestRatio (%),RestRestlessness,RestPerBout (minutes),ActivityDeviation (%),label
0,1,22,2018-08-01,0,58,1,3,52840,93.0,3.32,...,56623.962,139,613,12,2201,43,26,51,-4.0,0
1,1,22,2018-08-05,0,62,1,3,54376,95.0,3.10,...,7516.227,138,499,10,2265,35,34,50,4.0,0
2,1,22,2018-08-09,0,66,1,3,58753,95.0,3.12,...,13458.609,166,498,13,2448,34,37,38,16.0,0
3,1,22,2018-08-13,0,70,1,3,57293,96.0,3.23,...,311.171,141,733,10,2387,53,20,73,1.0,0
4,1,22,2018-08-17,0,74,1,3,54363,95.0,3.67,...,30974.205,139,613,10,2265,42,26,61,2.0,0


In [0]:
#write data into the file with label
data_set_new.to_csv("./CS5412_Cloud_Computing/Digital Agriculture/row_data_ID.json",index=False,sep=',')


In [60]:

# Read data from the file
data_new = pd.read_csv("./CS5412_Cloud_Computing/Digital Agriculture/row_data_ID.csv")
# data_new = shuffle(data_new, random_state = 555)
data_new.head()

,Animal_ID,Group_ID,Date,Dry_Days,Days_in_Milk,Is_Milked,Lactation_Num,Yield (gr),Conductivity,Fat (%),...,SCC (*1000/ml),Activity (steps/hour),RestTime (minutes),RestBout (#),ProdRate (gr/hour),RestRatio (%),RestRestlessness,RestPerBout (minutes),ActivityDeviation (%),label
0,1,22,2018-08-01,0,58,1,3,52840,93.0,3.32,...,56623.962,139,613,12,2201,43,26,51,-4.0,0
1,1,22,2018-08-05,0,62,1,3,54376,95.0,3.10,...,7516.227,138,499,10,2265,35,34,50,4.0,0
2,1,22,2018-08-09,0,66,1,3,58753,95.0,3.12,...,13458.609,166,498,13,2448,34,37,38,16.0,0
3,1,22,2018-08-13,0,70,1,3,57293,96.0,3.23,...,311.171,141,733,10,2387,53,20,73,1.0,0
4,1,22,2018-08-17,0,74,1,3,54363,95.0,3.67,...,30974.205,139,613,10,2265,42,26,61,2.0,0


In [61]:
# sort the data by date
data_new.sort_values(by=['Date'], ascending=True, inplace=True) 
data_new.head()

,Animal_ID,Group_ID,Date,Dry_Days,Days_in_Milk,Is_Milked,Lactation_Num,Yield (gr),Conductivity,Fat (%),...,SCC (*1000/ml),Activity (steps/hour),RestTime (minutes),RestBout (#),ProdRate (gr/hour),RestRatio (%),RestRestlessness,RestPerBout (minutes),ActivityDeviation (%),label
3865,8007,12,2018-04-03,0,73,1,4,49917,95.0,3.05,...,3427.676,157,589,8,2079,41,29,74,19.0,0
6898,9372,5,2018-04-03,0,63,1,2,50006,94.0,3.23,...,2870.781,149,-1,-1,2083,-1,-1,-1,5.0,1
4024,8106,22,2018-04-03,0,99,1,4,53204,92.0,2.81,...,46131.761,143,913,9,2216,59,20,101,4.0,0
11319,11181,8,2018-04-03,0,90,1,1,41281,94.0,2.50,...,6998.424,197,655,16,1720,46,31,41,18.0,0
5310,8731,23,2018-04-03,0,185,1,2,44598,105.0,2.19,...,31768.730,144,709,10,1858,49,23,71,12.0,0


In [0]:
n = int(data_new.shape[0])
data_1 = data_new.iloc[:n, :-1]
data_2 = data_new.iloc[n:n*2, :-1]
data_3 = data_new.iloc[n*2:, :-1]

In [0]:
data_1.to_csv("./CS5412_Cloud_Computing/Digital Agriculture/row_data_1.csv",index=False,sep=',')
data_2.to_csv("./CS5412_Cloud_Computing/Digital Agriculture/row_data_2.csv",index=False,sep=',')
data_3.to_csv("./CS5412_Cloud_Computing/Digital Agriculture/row_data_3.csv",index=False,sep=',')

In [62]:
#split the data into three files
n = data_new.shape[0]
print(n)
data_11 = data_new.iloc[0:1, :-1]
data_21 = data_new.iloc[1:2, :-1]
data_31 = data_new.iloc[2:3:, :-1]

for i in range(4,data_new.shape[0]):
    if i % 3 == 0:
        data_11 = data_11.append(data_new.iloc[i:i+1, :-1],ignore_index=True)   
    if i % 3 == 1:
        data_21 = data_21.append(data_new.iloc[i:i+1, :-1],ignore_index=True)  
    if i %3 == 2:
        data_31 = data_31.append(data_new.iloc[i:i+1, :-1],ignore_index=True)
        

14132


In [63]:
#write the data into three files
data_11.to_csv("./CS5412_Cloud_Computing/Digital Agriculture/row_data_1.csv",index=False,sep=',')
data_21.to_csv("./CS5412_Cloud_Computing/Digital Agriculture/row_data_2.csv",index=False,sep=',')
data_31.to_csv("./CS5412_Cloud_Computing/Digital Agriculture/row_data_3.csv",index=False,sep=',')
data_11.head()


,Animal_ID,Group_ID,Date,Dry_Days,Days_in_Milk,Is_Milked,Lactation_Num,Yield (gr),Conductivity,Fat (%),...,Blood (%),SCC (*1000/ml),Activity (steps/hour),RestTime (minutes),RestBout (#),ProdRate (gr/hour),RestRatio (%),RestRestlessness,RestPerBout (minutes),ActivityDeviation (%)
0,8007,12,2018-04-03,0,73,1,4,49917,95.0,3.05,...,0.11,3427.676,157,589,8,2079,41,29,74,19.0
1,9599,5,2018-04-03,0,59,1,2,45722,96.0,3.42,...,0.08,20606.305,285,-1,-1,1905,-1,-1,-1,24.0
2,8789,22,2018-04-03,0,71,1,3,55359,98.0,2.53,...,0.10,13182.563,223,670,4,2306,47,33,168,16.0
3,11095,8,2018-04-03,0,90,1,1,48751,105.0,2.59,...,0.14,363.078,206,595,15,2031,40,36,40,-3.0
4,835,23,2018-04-03,0,188,1,3,49086,95.0,3.10,...,0.15,4111.495,121,729,13,2045,49,18,56,33.0


In [58]:
data_11.shape

(4710, 23)

In [0]:
dataset_final[1].shape

(21,)

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [0]:
data = pd.read_csv('./row_data_sbsb.csv')
data.drop(labels=None,axis=1, index=None, columns="Dry_Days", inplace=True)

In [0]:
data = data.apply(pd.to_numeric, errors = "coerce")
# for s in data.iloc[0,:]:
#     print(type(s))

In [0]:
# print(data.shape)
# # data.replace(,np.nan)
# data.fillna(999999)
# data.dropna()
# print(data.shape)
# dry_day = data["Dry_Days"].unique()
# print(dry_day)

(14132, 20)
(14132, 20)
[0]


In [0]:
# for s in range(data.shape[0]):
#     for i in range(data.shape[1]):
#         data.iloc[s:s+1,i:i+1] = float(data.iloc[s:s+1,i:i+1])
#     print(type(s))

TypeError: ignored

In [0]:
X = data.iloc[:, :-1]
y = data.iloc[:, -1]
X

le = LabelEncoder()
y = le.fit_transform(y)
X_trn, X_tst, y_trn, y_tst = train_test_split(X, y, test_size=0.2, random_state=42)
# y_trn
for s in X_trn.iloc[1:2]:
    print(s)
print(X.shape)
y.shape

Days_in_Milk
Is_Milked
Lactation_Num
Yield (gr)
Conductivity
Fat (%)
Protein (%)
Lactose (%)
Blood (%)
SCC (*1000/ml)
Activity (steps/hour)
RestTime (minutes)
RestBout (#)
ProdRate (gr/hour)
RestRatio (%)
RestRestlessness
RestPerBout (minutes)
ActivityDeviation (%)
(14132, 18)


(14132,)

In [0]:
##Bagging and Forest Tree
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

clf = DecisionTreeClassifier(criterion = "entropy", max_depth = 5)
model = clf.fit(X_trn, y_trn)
y_pre_tst = clf.predict(X_tst)
y_pre_trn = clf.predict(X_trn)
Err_Train = accuracy_score(y_pre_trn, y_trn)
Err_Test = accuracy_score(y_pre_tst, y_tst)
print(Err_Train, Err_Test)

ValueError: ignored